In [57]:
import pandas as pd
import numpy as np

In [58]:
df=pd.read_csv("E:/git/dsmp-2024-group22/data_preprocess/DATA/processed_tapes.csv",index_col=0)
# df=df.drop("Unnamed: 0",axis=1)
df=df.dropna()
df=df.sort_values(by='date')
df.rename(columns={'max_price': 'High', 'min_price': 'Low','open_price': 'Open','close_price': 'Close'}, inplace=True)
df.Open

0      267.0
1      281.0
4      278.0
5      336.0
6      365.0
       ...  
174     96.0
175    113.0
176    106.0
179    103.0
180     94.0
Name: Open, Length: 125, dtype: float64

In [59]:
df_predict=pd.read_csv("E:/git/dsmp-2024-group22/data_preprocess/DATA/LSTM_prediction.csv")
df_predict['Open'] = pd.to_numeric(df_predict['Open'], downcast='float')
df_predict['High'] = pd.to_numeric(df_predict['High'], downcast='float')
df_predict['Low'] = pd.to_numeric(df_predict['Low'], downcast='float')
df_predict

,Unnamed: 0,Open,High,Low,Close
0,2025-04-28-1,158.0,160.0,140.0,152.052260
1,2025-04-28-2,152.0,161.0,144.0,152.442180
2,2025-04-28-3,158.0,162.0,142.0,154.090730
3,2025-04-28-4,161.0,162.0,141.0,155.025620
4,2025-04-28-5,160.0,160.0,140.0,155.110200
...,...,...,...,...,...
2245,2025-07-01-46,109.0,113.0,93.0,110.052704
2246,2025-07-01-47,109.0,113.0,93.0,109.887670
2247,2025-07-01-48,109.0,113.0,93.0,109.707230
2248,2025-07-01-49,107.0,113.0,93.0,109.346500


In [61]:
df_v=pd.read_csv("E:\git\dsmp-2024-group22\data_preprocess\DATA\sperated_tapes.csv")
df_predict["Volume"]=df_v["Volume"]
df_predict

,Unnamed: 0,Open,High,Low,Close,Volume
0,2025-04-28-1,158.0,160.0,140.0,152.052260,955
1,2025-04-28-2,152.0,161.0,144.0,152.442180,908
2,2025-04-28-3,158.0,162.0,142.0,154.090730,936
3,2025-04-28-4,161.0,162.0,141.0,155.025620,978
4,2025-04-28-5,160.0,160.0,140.0,155.110200,983
...,...,...,...,...,...,...
2245,2025-07-01-46,109.0,113.0,93.0,110.052704,1032
2246,2025-07-01-47,109.0,113.0,93.0,109.887670,1103
2247,2025-07-01-48,109.0,113.0,93.0,109.707230,1098
2248,2025-07-01-49,107.0,113.0,93.0,109.346500,1087


In [62]:
print(df_predict.head())

     Unnamed: 0   Open   High    Low      Close  Volume
0  2025-04-28-1  158.0  160.0  140.0  152.05226     955
1  2025-04-28-2  152.0  161.0  144.0  152.44218     908
2  2025-04-28-3  158.0  162.0  142.0  154.09073     936
3  2025-04-28-4  161.0  162.0  141.0  155.02562     978
4  2025-04-28-5  160.0  160.0  140.0  155.11020     983


In [64]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


print(GOOG.head())



              Open    High     Low   Close    Volume
2004-08-19  100.00  104.06   95.96  100.34  22351900
2004-08-20  101.01  109.08  100.50  108.31  11428600
2004-08-23  110.75  113.48  109.05  109.40   9137200
2004-08-24  111.24  111.60  103.57  104.87   7631300
2004-08-25  104.96  108.00  103.88  106.00   4598900


In [68]:
df_predict

,Unnamed: 0,Open,High,Low,Close,Volume
0,2025-04-28-1,158.0,160.0,140.0,152.052260,955
1,2025-04-28-2,152.0,161.0,144.0,152.442180,908
2,2025-04-28-3,158.0,162.0,142.0,154.090730,936
3,2025-04-28-4,161.0,162.0,141.0,155.025620,978
4,2025-04-28-5,160.0,160.0,140.0,155.110200,983
...,...,...,...,...,...,...
2245,2025-07-01-46,109.0,113.0,93.0,110.052704,1032
2246,2025-07-01-47,109.0,113.0,93.0,109.887670,1103
2247,2025-07-01-48,109.0,113.0,93.0,109.707230,1098
2248,2025-07-01-49,107.0,113.0,93.0,109.346500,1087


In [69]:
bt = Backtest(df_predict, SmaCross,
              cash=1000000, commission=.002,
              exclusive_orders=True)

output = bt.run()
output

C:\Users\14620\AppData\Local\Temp/ipykernel_4312/1713374783.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_predict, SmaCross,


Start                                     0.0
End                                    2249.0
Duration                               2249.0
Exposure Time [%]                        96.8
Equity Final [$]                  2930130.886
Equity Peak [$]                4256659.184648
Return [%]                         193.013089
Buy & Hold Return [%]              -29.408853
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -35.083754
Avg. Drawdown [%]                   -5.674938
Max. Drawdown Duration                  554.0
Avg. Drawdown Duration              36.457627
# Trades                                183.0
Win Rate [%]                        46.994536
Best Trade [%]                      18.954011
Worst Trade [%]                    -16.134118
Avg. Trade [%]                    

In [70]:
bt.plot()

Row(id='8318', ...)